In [17]:
import pandas as pd
import sqlite3

create a connection to the database using the library sqlite3


In [18]:
con = sqlite3.connect("../data/checking-logs.sqlite.sqlite_copy")

In [19]:
# просто напомнить себе как выглядит таблица
query = '''
SELECT * FROM checker LIMIT 10'''

pd.io.sql.read_sql(query, con)

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [20]:
query = '''
SELECT * FROM pageviews LIMIT 10'''

pd.io.sql.read_sql(query, con)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


create a new table datamart in the database 

by joining the tables pageviews and checker using only one query

    ◦ the table should have the following columns: uid, labname, first_commit_ts, first_view_ts

    ◦ first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user

    ◦ first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed

    ◦ status = ’ready’ should still be a filter

    ◦ numTrials = 1 should still be a filter

    ◦ labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
    
    
    ◦ the table should contain only the users (uids with user_*) and not the admins

In [21]:
query = '''
SELECT checker.uid,
       checker.labname,
       checker.timestamp AS first_commit_ts,
       pageviews.datetime AS first_view_ts
FROM checker 
LEFT JOIN pageviews ON checker.uid=pageviews.uid
WHERE status='ready'
AND numTrials=1
AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND checker.uid LIKE 'user_%'
AND (pageviews.datetime = (SELECT MIN(pageviews.datetime)
                           FROM pageviews
                           WHERE uid=checker.uid)
        OR pageviews.datetime IS NULL)
'''
datamart = pd.io.sql.read_sql(query, con, parse_dates=[
                              'first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


## using Pandas methods, create two dataframes: test and control

    ◦ test should have the users that have the values in first_view_ts
    ◦ control should have the users that have missing values in first_view_ts

In [22]:
test = datamart[~datamart['first_view_ts'].isnull()]
test.head()

,uid,labname,first_commit_ts,first_view_ts
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


    ◦ replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis

In [23]:
control = datamart[datamart['first_view_ts'].isnull()]
control = control.fillna(test['first_view_ts'].mean())
control

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,2020-04-27 00:40:05.761783552
1,user_4,laba04,2020-04-17 11:33:17.366400,2020-04-27 00:40:05.761783552
2,user_4,laba04s,2020-04-17 11:48:41.992466,2020-04-27 00:40:05.761783552
5,user_2,laba04,2020-04-18 13:42:35.482008,2020-04-27 00:40:05.761783552
6,user_2,laba04s,2020-04-18 13:51:22.291271,2020-04-27 00:40:05.761783552
...,...,...,...,...
126,user_2,laba06s,2020-05-19 14:45:03.908268,2020-04-27 00:40:05.761783552
132,user_6,laba06s,2020-05-20 14:50:07.609937,2020-04-27 00:40:05.761783552
134,user_7,laba06s,2020-05-20 23:05:37.742597,2020-04-27 00:40:05.761783552
135,user_23,laba06,2020-05-21 08:34:10.517205,2020-04-27 00:40:05.761783552


    ◦ save both tables into the database, you will use them in the next exercises

In [24]:
test.to_sql('test', con)
control.to_sql('control', con)

ValueError: Table 'test' already exists.

    close the connection


In [ ]:
con.close()